In [126]:
import pandas as pd
import math

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import re
import csv

In [127]:
ib=pd.DataFrame([])
cgmm=pd.DataFrame([])
cgm0=pd.DataFrame([])

for i in range(1,6):
    tmpib=pd.read_csv('InsulinBolusLunchPat'+str(i)+'.csv')
    ib[i]=tmpib.max(axis=1,skipna=True)
    
    tmpcgm=pd.read_csv('CGMSeriesLunchPat'+str(i)+'.csv')
    cgmm[i]=tmpcgm.max(axis=1,skipna=True)
    
    cgm0[i]=pd.read_csv("CGMSeriesLunchPat"+str(i)+".csv")['cgmSeries_25']
    
ib=ib.fillna(ib.mean(axis=1).T, axis=0)
cgmm=cgmm.fillna(cgmm.mean(axis=1).T, axis=0)
cgm0 = cgm0.fillna(cgm0.mean(axis=1).T, axis=0)

In [128]:
def quantize(x):
    if x%10!=0:
        x=math.modf((x-40)/10)[1]+1
    else:
        x=math.modf((x-40)/10)[1]
    return int(x)

In [129]:
cgmm=cgmm.applymap(lambda x: quantize(x))

In [130]:
cgm0=cgm0.applymap(lambda x: quantize(x))

In [131]:
def output(filename,x):
    with open(filename+'.csv','a+',newline ='') as file:
        write = csv.writer(file) 
        for y in x:
            write.writerows([y]) 

In [132]:
for c in range(5):
    data_list = []
    for i in range(33):
        l = []
        l.append("a"+str(cgmm.values[i,c]))
        l.append("b"+str(cgm0.values[i,c]))
        l.append("c"+str(ib.values[i,c]))
        data_list.append(l)
    
    te=TransactionEncoder()
    te_arr=te.fit_transform(data_list)
    te_df=pd.DataFrame(te_arr, columns=te.columns_)
    
    frequent=apriori(te_df,min_support= 0.000001,use_colnames=True)
    frequent['length']=frequent['itemsets'].apply(lambda x: len(x))
    
    freq3=frequent[(frequent['length']==3)&(frequent['support']>=0.00001)]
    freq3=freq3.sort_values('support',ascending=False)
    
    freq_itemsets = freq3['itemsets'][:7] 
    
    l=[sorted(list(x)) for x in freq_itemsets]
    w=[]
    for x in l:
        w.append(tuple([re.sub('[a-z]', '', y) for y in x]))
    output('frequent_itemsets',w)
        
    rule_1=association_rules(frequent,metric="confidence",min_threshold=0.0001)
    rule_1["antecedent_len"]=rule_1["antecedents"].apply(lambda x: len(x))
    
    rule1_max=rule_1[(rule_1['antecedent_len']>=2)&(rule_1['confidence']>0.99)&(rule_1['lift']>1.2)]
    rule1_max=rule1_max.sort_values('confidence',ascending=False)[:3]
    rule1_max_antecendent=[tuple(x) for x in rule1_max['antecedents']] 
    rule1_max_consequent=[tuple(x) for x in rule1_max['consequents']]
    
    combine=list(zip(rule1_max_antecendent,rule1_max_consequent))
    
    association1=[]
    for x in combine:
        fin=[]
        for y in x:
            fin+=y
        association1.append(fin)
        
    lis=[sorted(list(x)) for x in association1]
    rule1=[]
    for x in lis:
        for y in x:
            tmp=list([re.sub('[a-z]', '', y) for y in x])
        rule1.append(["{"+str(tmp[0])+","+str(tmp[1])+"-->"+str(tmp[2])+"}"])
    output('bestConfidence_rule1',rule1)
    
    rule2_min=rule_1[(rule_1['antecedent_len']>=2)&(rule_1['confidence']<0.6)&(rule_1['lift']>1.2)]
    rule2_min=rule2_min.sort_values('confidence',ascending=True)[:1]
    
    combine=list(zip(tuple(rule2_min['antecedents']),tuple(rule2_min['consequents'])))
    
    association2=[]
    for x in combine:
        fin=[]
        for y in x:
            fin+=y
        association2.append(fin)
        
    lis=[sorted(list(x)) for x in association1]
    rule2=[]
    for x in lis:
        for y in x:
            tmp=list([re.sub('[a-z]', '', y) for y in x])
        rule2.append(["{"+str(tmp[0])+","+str(tmp[1])+"-->"+str(tmp[2])+"}"])
    output('leastConfidence_rule2',rule2)